In [6]:
import numpy as np

class Bins:

    def __init__(self, start: float, count: int, step: float):

        assert count > 0 and step > 0
        
        self.thresholds = np.arange(count).astype(float) * step + start
            
        self.start = start
        self.count = count
        self.step = step
        self.end = self.thresholds[-1]

    def convert(self, X: np.ndarray):
        """
        Transform a continuous value X into an N-dimensional vector p based on given thresholds.
        This function handles arrays X of any shape and a 1D array of thresholds.
        
        The transformation is as follows:
        - If x < th0, then p[0] = 1.
        - If x > th(N-1), then p[N-1] = 1.
        - If th[i] < x < th[i+1], then p[i] = (th[i+1] - x) / s and p[i+1] = (x - th[i]) / s,
          where s is the step size (assumed to be constant between thresholds).
        
        The function uses numpy operations to efficiently process the transformation
        without explicit looping.
    
        Parameters:
        - X : ndarray
            An array of any shape containing the continuous values to be transformed.
    
        Returns:
        - p_vectors : ndarray
            An array of shape (..., N) where each "row" corresponds to the transformed
            vector for each element in X. Each vector p has the property that sum(p) = 1.
    
        Note:
        This function assumes that the thresholds are equally spaced.
        """

        # Calculate normalized distances for each threshold
        distances = (X[..., None] - self.thresholds) / self.step
        
        # Apply clipping to handle boundary conditions
        distances_clipped = np.clip(distances, -1, 1)
        
        # Calculate p vectors
        p_vectors = np.maximum(0, 1 - np.abs(distances_clipped))
        
        # Ensure sum(p) equals 1 (handling boundary conditions)
        p_vectors /= p_vectors.sum(axis=-1, keepdims=True)

        return p_vectors
        
    def revert(self, p_vectors: np.ndarray):
    
        """
        Revert the transformed p_vectors back to the original continuous values X.
        This function assumes the input p_vectors are obtained from the previous transformation
        and uses the provided thresholds to revert the transformation.
    
        Parameters:
        - p_vectors : ndarray
            An array of shape (..., N) containing transformed vectors, where each vector p
            has the property that sum(p) = 1.
    
        Returns:
        - X_reverted : ndarray
            An array of the original continuous values from which the p_vectors were derived.
            The shape of X_reverted is the same as the shape of p_vectors, except for the last dimension.
    
        Note:
        This function assumes that the thresholds are equally spaced and that the p_vectors
        are correctly formatted (sum to 1 for each vector).
        """
    
        # Calculate the weighted sum of the thresholds
        X_reverted = np.sum(p_vectors * self.thresholds, axis=-1)
    
        return X_reverted

In [7]:
X = np.arange(10) * 0.067 + 0.4
X

array([0.4  , 0.467, 0.534, 0.601, 0.668, 0.735, 0.802, 0.869, 0.936,
       1.003])

In [8]:
bins = Bins(start=0.5, count=3, step=0.2)
bins.thresholds

array([0.5, 0.7, 0.9])

In [9]:
bins.convert(X)

array([[1.   , 0.   , 0.   ],
       [1.   , 0.   , 0.   ],
       [0.83 , 0.17 , 0.   ],
       [0.495, 0.505, 0.   ],
       [0.16 , 0.84 , 0.   ],
       [0.   , 0.825, 0.175],
       [0.   , 0.49 , 0.51 ],
       [0.   , 0.155, 0.845],
       [0.   , 0.   , 1.   ],
       [0.   , 0.   , 1.   ]])

In [10]:
bins.revert(bins.convert(X))

array([0.5  , 0.5  , 0.534, 0.601, 0.668, 0.735, 0.802, 0.869, 0.9  ,
       0.9  ])